In [ ]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras

import warnings
warnings.filterwarnings("ignore")

df_ft = pd.read_csv('features_n_target.csv')
df_ft.columns.to_list()

In [ ]:
features = [
 'monetary_DNN',
 'frequency_DNN',
 'frequency',
 't',
 'time_between',
 'avg_basket_value',
 'avg_basket_size',
 'cnt_returns',
 'has_returned']
numeric_f = ['monetary_DNN','frequency_DNN','frequency','t',
              'time_between','avg_basket_value','avg_basket_size','cnt_returns']
cate_f = ['has_returned']

target = ['target_monetary']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X_train_full,X_test,y_train_full,y_test = train_test_split(df_ft[features],df_ft[target],test_size = 0.2,random_state = 42)
X_train,X_valid,y_train,y_valid = train_test_split(X_train_full,y_train_full,test_size = 0.15, random_state = 42)

scaler = StandardScaler()
# X_train_full = scaler.fit_transform(X_train_full)
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

train_num,train_cate = X_train[:,:-1],X_train[:,-1]
valid_num,valid_cate = X_valid[:,:-1],X_valid[:,-1]
test_num,test_cate = X_test[:,:-1],X_test[:,-1]

# Fine tuning 

### Talos  

In [ ]:
def widendeep_model(x_train,y_train,x_valid,y_valid,params):

    input_num = tf.keras.layers.Input(shape = (8,),name = 'deep_input')
    input_cate= tf.keras.layers.Input(shape = (1,),name = 'wide_input')


    hidden_= tf.keras.layers.Dense(params['units'],
                           kernel_initializer = params['initial'], 
                           kernel_regularizer=tf.keras.regularizers.l2(0.01))(input_num)
    hidden_ = tf.keras.layers.BatchNormalization()(hidden_)
    hidden_ = tf.keras.activations.relu(hidden_)
    
    
    hidden_ = tf.keras.layers.Dense(params['units'],
                       kernel_initializer = params['initial'], 
                       kernel_regularizer=tf.keras.regularizers.l2(0.01))(hidden_)
    hidden_ = tf.keras.layers.BatchNormalization()(hidden_)
    hidden_ = tf.keras.activations.relu(hidden_)
    hidden_ = tf.keras.layers.Dropout(params['dropout'])(hidden_)


    concat = tf.keras.layers.concatenate([input_cate,hidden_])
    output = tf.keras.layers.Dense(1,name = 'output')(concat)

    model = tf.keras.Model(inputs = [input_cate,input_num],outputs=[output])
    optimizer = tf.keras.optimizers.Adam(learning_rate = params['learning_rate'],beta_1=0.9, beta_2=0.999)
    metrics = tf.keras.metrics.RootMeanSquaredError()
    model.compile(optimizer = optimizer,loss='mse',metrics=metrics)

    out = model.fit(x= x_train,
                    y= y_train,
                    validation_data=[x_valid,y_valid],
                    epochs=150,
                    batch_size=params['batch_size'],
                    verbose=0)
    
    return out,model

In [ ]:
import talos
# from keras.utils import plot_model
from scipy.stats import reciprocal
input_shape=[8,1]
learning_rate=reciprocal(3e-4,3e-2).rvs(100).tolist()
units = [128,64,32,16]
initial =['lecun_normal','glorot_uniform','he_normal']
batch_size=[12,16,20]
dropout = [0.1,0.15,0.2,0.25]

params = {'units':units,
         'initial':initial,
         'learning_rate':learning_rate,
         'batch_size':batch_size,
         'dropout':dropout} 


scan_object = talos.Scan(x=[X_train[:,8].reshape(-1,1),X_train[:,:8]],
                         y=y_train,
                         x_val=[X_valid[:,8].reshape(-1,1),X_valid[:,:8]],
                         y_val=y_valid,
                         params=params,
                         model=widendeep_model,experiment_name='try')


In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

units_list = [[128,64],[64,64],[64,32],[32,32],[32,16],[16,16]]
learning_rates=reciprocal(3e-4,3e-2).rvs(100).tolist()
dropout =0.15
batch_size = 12
params_history = []
for units in units_list:
    for learning_rate in learning_rates:
        model = widendeep_model(units,learning_rate)
        history = model.fit((train_cate.reshape(-1,1),train_num), 
                            y_train, 
                            epochs=200, 
                            shuffle=True,
                            batch_size = batch_size,
                            validation_data = ((valid_cate.reshape(-1,1),valid_num),y_valid),
                            callbacks=[earlystop]
         )
        params_history.append([units,learning_rate,history.history['val_root_mean_squared_error']])

# Get optimized parameters

In [ ]:
df = pd.DataFrame(params_history,columns=['units','learning_rate','loss'])
min_loss = np.zeros(len(df))
for i in range(len(df)):
    loss = min(df.loss[i])
    min_loss[i] = loss
inx = np.argmin(min_loss)
print('optimal units:',df.iloc[inx,0],
     '\noptimal learning rate:',df.iloc[inx,1])

# Train the multiple imput model

In [ ]:
# import keras_tuner as kt
# from keras_tuner import RandomSearch
from scipy.stats import reciprocal


def widendeep_model(units,learning_rate):

    input_num = tf.keras.layers.Input(shape = [8],name = 'deep_input')
    input_cate= tf.keras.layers.Input(shape = [1],name = 'wide_input')


    hidden_= tf.keras.layers.Dense(units[0],
                           kernel_initializer = 'lecun_normal', 
                           kernel_regularizer=tf.keras.regularizers.l2(0.01))(input_num)
    hidden_ = tf.keras.layers.BatchNormalization()(hidden_)
    hidden_ = tf.keras.activations.relu(hidden_)
    
    
    hidden_ = tf.keras.layers.Dense(units[1],
                       kernel_initializer ='lecun_normal', 
                       kernel_regularizer=tf.keras.regularizers.l2(0.01))(hidden_)
    hidden_ = tf.keras.layers.BatchNormalization()(hidden_)
    hidden_ = tf.keras.activations.relu(hidden_)
    hidden_ = tf.keras.layers.Dropout(dropout)(hidden_)


    concat = tf.keras.layers.concatenate([input_cate,hidden_])
    output = tf.keras.layers.Dense(1,name = 'output')(concat)

    model = tf.keras.Model(inputs = [input_cate,input_num],outputs=[output])
    optimizer = tf.keras.optimizers.Adam(learning_rate,beta_1=0.9, beta_2=0.999)
    metrics = tf.keras.metrics.RootMeanSquaredError()
    model.compile(optimizer = optimizer,loss='mse',metrics=metrics)
    
    return model

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

# tensorboard = keras.callbacks.TensorBoard(run_logdir)


earlystop = keras.callbacks.EarlyStopping(patience=100,
                                          restore_best_weights=True)

units = [64, 64]
learning_rate = 0.00232087319077393
dropout =0.15
batch_size = 12

model = widendeep_model(units,learning_rate)
model.fit((train_cate.reshape(-1,1),train_num), y_train, epochs=300, shuffle=True,
          batch_size = batch_size,
                    validation_data = ((valid_cate.reshape(-1,1),valid_num),y_valid),
                     callbacks=[earlystop]
         )
mse_test = model.evaluate((test_cate.reshape(-1,1),test_num), y_test) 



In [ ]:
print('Root Mean Squared Error:', mse_test)